In [1]:
import pandas as pd
import numpy as np
import re

# recurrence relation
$$\vec{p}_n = \vec{p}_0 + n\vec{v}_0 + \sum_{i=1}^{n}i\vec{a}_0$$

$$\vec{p}_n = \vec{p}_0 + n\vec{v}_0 +\frac{n(n+1)}{2}\vec{a}_0$$

In [2]:
test_data='''p=< 3,0,0>, v=< 2,0,0>, a=<-1,0,0>
p=< 4,0,0>, v=< 0,0,0>, a=<-2,0,0>'''.splitlines()
real_data = open('input.txt').readlines()

In [3]:
parser = re.compile(r'(.=<([^,]+),([^,]+),([^,])+>(, )?)(.=<([^,]+),([^,]+),([^,])+>(, )?)(.=<([^,]+),([^,]+),([^,])+>(, )?)')

In [4]:
def parse_lines(lines):
    p = []
    v = []
    a = []
    for line in lines:
        l = parser.match(line)
        if l:
            p.append(list(map(int,l.group(2, 3, 4))))
            v.append(list(map(int,l.group(7, 8, 9))))
            a.append(list(map(int,l.group(12, 13, 14))))
    return map(lambda x: np.array(x, dtype=np.int64), [p, v, a]) # , dtype=np.float64

In [5]:
def manhattan(v):
    return abs(v).sum(axis=1)

def norm2(v):
    return (v**2).sum(axis=1)

In [6]:
# p, v, a = parse_lines(real_data)
p0, v0, a0 = parse_lines(real_data)
# n = 100000
# for i in range(n):
#     v = v + a
#     p = p + v

In [7]:
# def p_i(n):
#     return p0 + n*v0 + (n*(n+1)*a)/2

# np.where(manhattan(p_i(n*1000000)) == manhattan(p_i(n*1000000)).min())

# Part 1 - doesn't acceleration just win out?

particles `[ 21, 159, 285, 442, 457]` have the equal lowest acceleration.

particle 457 starts nearest the origin and moves slowest at the beginning (all of these particles are have a component of their velocity in the same direction as their respective accelerations, i.e. none is currently slowing)

therefore the part 1 answer is 457

In [9]:
test_data2 = '''p=<-6,0,0>, v=< 3,0,0>, a=< 0,0,0>    
p=<-4,0,0>, v=< 2,0,0>, a=< 0,0,0>
p=<-2,0,0>, v=< 1,0,0>, a=< 0,0,0>
p=< 3,0,0>, v=<-1,0,0>, a=< 0,0,0>'''.splitlines()

In [10]:
#p0, v0, a0 = parse_lines(test_data2)
p0, v0, a0 = parse_lines(real_data)

## separate dataframes for $\vec{p}$, $\vec{v}$ and $\vec{a}$

In [11]:
iters = 0
p, v, a = map(pd.DataFrame, [p0, v0, a0])
lastcount = len(p) + 1

In [12]:
while iters < 100:
    if iters % 10000 == 0:
        print(f'iteration {iters:8}')
    v = v + a
    p = p + v
    mask = ~p.duplicated(keep=False)
    v = v[mask].reset_index(drop=True)
    a = a[mask].reset_index(drop=True)
    p = p[mask].reset_index(drop=True)
    count = len(p)
    iters += 1
    if count != lastcount:
        print(f'iteration {iters:8}: count = {count}', flush=True)
        lastcount = count



iteration        0
iteration        1: count = 1000
iteration       10: count = 998
iteration       18: count = 996
iteration       35: count = 994
iteration       36: count = 992


In [13]:
len(p)

992

particles `[ 21, 159, 285, 442, 457]` have the equal lowest acceleration.

particle 457 starts nearest the origin and moves slowest at the beginning

therefore the part 1 answer is 457

## single dataframe for $\vec{p}$, $\vec{v}$ and $\vec{a}$

In [88]:
p0, v0, a0 = parse_lines(real_data)


sdf = pd.concat(map(pd.DataFrame, [p0, v0, a0]), axis=1, ignore_index=True)
V = ['v0', 'v1', 'v2']
A = ['a0', 'a1', 'a2']
P = ['p0', 'p1', 'p2']

sdf.columns = P + V + A


In [89]:
sdf.head()

,p0,p1,p2,v0,v1,v2,a0,a1,a2
0,-833,-499,1,84,17,1,-4,1,1
1,-168,3586,1,-61,-58,1,7,-13,8
2,364,223,7,31,-11,1,-5,0,3
3,769,-854,5,-20,4,4,0,1,9
4,6985,-3666,3,-112,99,3,-4,0,4


In [90]:
from IPython.display import display, HTML
iters = 0
lastcount = len(sdf)+1 # set higher than he length, so that the count will be displayed on the first iteration


In [91]:
#display(sdf)
while iters < 100:
    sdf.loc[:, V] += sdf.loc[:, A].rename(columns=dict(zip(A, V)))
    sdf.loc[:, P] += sdf.loc[:, V].rename(columns=dict(zip(V, P)))
    #display(sdf)
    mask = ~sdf.duplicated(subset=P, keep=False) # keep = False == don't include ANY dupes in output
    if sum(~mask) != 0:
        display(HTML('Collision between particles:'))
        display(sdf[~mask])
    sdf = sdf[mask].reset_index(drop=True)
    count = len(sdf)
    iters += 1
    if count != lastcount:
        print(f'iteration {iters:8}: count = {count}', flush=True)
        lastcount = count


iteration        1: count = 1000


,p0,p1,p2,v0,v1,v2,a0,a1,a2
395,13,-27,530,106,-185,89,4,-17,8
401,13,-27,530,-103,-6,89,-2,-11,8


iteration       10: count = 998


,p0,p1,p2,v0,v1,v2,a0,a1,a2
306,30,-14,728,-9,182,74,1,14,4
314,30,-14,728,-144,12,74,-12,5,4


iteration       18: count = 996


,p0,p1,p2,v0,v1,v2,a0,a1,a2
467,-8,-15,700,-338,-361,37,-11,-7,1
469,-8,-15,700,547,231,37,15,6,1


iteration       35: count = 994


,p0,p1,p2,v0,v1,v2,a0,a1,a2
212,-49,33,72,-299,72,2,-8,4,0
213,-49,33,72,-278,-148,2,-9,-1,0


iteration       36: count = 992


In [61]:
iters

100

In [62]:
mask.sum()

992

In [63]:
sdf.loc[:, V] = sdf.loc[:, V] + sdf.loc[:, A].rename(columns=dict(zip(A, V)))
sdf.loc[:, P] = sdf.loc[:, P] + sdf.loc[:, V].rename(columns=dict(zip(V, P)))
#display(sdf)
mask = ~sdf.duplicated(subset=P, keep=False) # keep = False == don't include ANY dupes in output


In [64]:
mask.sum()

992

In [65]:
sdf[~mask]

,p0,p1,p2,v0,v1,v2,a0,a1,a2


In [66]:
sdf[460:475]

,p0,p1,p2,v0,v1,v2,a0,a1,a2
460,8189,-38966,31415,90,-788,611,0,-8,6
461,12648,-10301,36874,319,-248,715,4,-3,7
462,12648,-24497,26673,229,-449,514,2,-4,5
463,8826,-14214,36773,97,-156,714,0,0,7
464,-89636,1347,11114,-1705,15,210,-16,0,2
465,223,17376,30906,36,361,606,1,3,6
466,45202,-6087,11014,1010,-92,209,10,0,2
467,-23933,1437,6065,-525,22,110,-5,0,1
468,57940,-6054,606,1333,-189,6,14,-3,0
469,-22547,33975,42122,-439,775,817,-3,8,8


In [67]:
iters += 1
sdf = sdf[mask].reset_index(drop=True)

In [68]:
len(sdf)

992

two particles $P$ and $P'$ with parameters $\{\vec{p}, \vec{v}, \vec{a}\}$ and $\{\vec{p'}, \vec{v'}, \vec{a'}\}$ collide at time n, if and only if the following are equal:

$$\vec{p}_n = \vec{p}_0 + n\vec{v}_0 +\frac{n(n+1)}{2}\vec{a}_0$$

$$\vec{p'}_n = \vec{p'}_0 + n\vec{v'}_0 +\frac{n(n+1)}{2}\vec{a'}_0$$

i.e. if

$$\vec{p}_0 + n\vec{v}_0 +\frac{n(n+1)}{2}\vec{a}_0 = \vec{p'}_0 + n\vec{v'}_0 +\frac{n(n+1)}{2}\vec{a'}_0$$

$$(\vec{p}_0 - \vec{p'}_0) + n(\vec{v}_0 - \vec{v'}_0) + \frac{n(n+1)}{2}(\vec{a}_0 -\vec{a'}_0) = 0$$